In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:14 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-28 23:33:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.86MB/s    in 0.2s    

2022-09-28 23:33:02 (5.86 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...|2015-08-31 00:00:00|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| 

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46263557|             1|
|   44418648|             1|
|   34220092|             1|
|   10770444|             1|
|   12819130|             1|
|   24692214|             3|
|   51981844|             3|
|   43622169|             1|
|   43097449|             1|
|   45978717|             1|
|   20227781|             1|
|   11487525|             4|
|   31169607|             1|
|   22552080|             2|
|   15829398|             2|
|   46909180|             1|
|     198573|             3|
|   33383395|             1|
|    2934525|             3|
|    2333626|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0081SBO4Y|Key Pair Lawn Wit...|
|B00EGFKOZ6|Swiftly Done Brig...|
|B00SOU5MYM|Apalus 4.2" LCD S...|
|B00GIDK4LS|Genuine Honda OEM...|
|B001GQTZ4E|Luster Leaf Rapic...|
|B000BQM5DS|Roebic Laboratori...|
|B00AUXEVFW|Sunset Flower- 25...|
|B006RYWT7K|Stens 390-040 Nyl...|
|B00KGCHQIG|    Grey Ground Caps|
|B00UE7H4GW|bbq factory® JBX5...|
|B00BJ8AAE8|Winter Squash, Su...|
|B00HUDTYSO|20W x 19Dx 2.5H S...|
|B007PZBARU|Hearth & Garden L...|
|B00IOVPUJS|Husqvarna HU625HW...|
|B004AWAXOM|30"x60' Mexican T...|
|B00BD2BA1C|Classic Accessori...|
|B00XYMHZ5Y|Pool Blaster Ecli...|
|B000RUAR0G|Sevin Lawn Insect...|
|B001GP85G4|Zodiac B5 Sweep H...|
|B009P8BW9G|Miniature Fairy G...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RED72VWWCOS7S|   32787517|B008HDQYLQ|     348668413| 2015-08-31|
| RZHWQ208LTEPV|   16374060|B005OBZBD6|     264704759| 2015-08-31|
|R37LBC3XAVLYOO|    9984817|B00RQL8U2G|      95173602| 2015-08-31|
|R3L7XJMA0MVJWC|   12635190|B0081SBO4Y|     835659279| 2015-08-31|
|R2I2GHSI7T1UBN|   43905102|B008E6OK3U|     539243347| 2015-08-31|
|R2GFFKHK4I6VMX|   52596997|B00W6NTULY|     337446474| 2015-08-31|
|R1R0UDX2XAN1S4|   43871104|B00GXUMYKA|     468857193| 2015-08-31|
|R22C8FMBSTFRY8|   11346008|B005EIX8JS|     125753094| 2015-08-31|
|R118NNIQ75XPGO|   49206471|B000HJBKMQ|     834273114| 2015-08-31|
|R30HYXHZQ49621|   37596267|B004LY59V6|     612086079| 2015-08-31|
|R3EMLKY0GF1E90|   31554283|B00CAVM85M|     280334010| 2015-08-31|
|R23BX7EGJMGQJR|   43211735|B00DP6X1LG|     233116679| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://{url}:5432/{table}"
config = {"user":"postgres", 
          "password": "{password}", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
# 8 min
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 1 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 2 min
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 8 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)